<a href="https://colab.research.google.com/github/supertime1/Afib_PPG/blob/master/Afib_ECG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

This notebook trains an ECG DNN by using labeled ECG data from "The PhysioNet Computing in Cardiology Challenge 2017" (https://physionet.org/content/challenge-2017/1.0.0/). The ECG DNN model will be used to label the ECG data from MIMIC-III waveform dataset, so as the concurrent PPG data.

The ECG data used in training and validation has the following important attributes:
*   sampling frequency: 300Hz
*   4 lables: Normal (N), AF (A), Other rhythm (O), Noisy (~)
*   length: 9 - 60s with 30s mean.
*   preprosessed: data has been band pass filtered by AliveCor device


Only time length >30s is used in training, since PPG data usually requires 30s for Afib detection.

#Import Data and Label

In [0]:
!pip install tensorflow-gpu==2.0.0

In [0]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import glob
import wfdb
import tensorflow as tf

In [0]:
hd_names = []
for name in glob.glob("C:/Users/57lzhang.US04WW4008/Desktop/Afib/Afib_ECG data/training2017/*.hea"):
  position = name.index('.hea')
  name = name[0:position] #remove the .hea part to comply the wfdb.rdrecord format
  hd_names.append(name)
len(hd_names)

In [0]:
#load label here? so that it can be dropped together later

In [0]:
qualified_names = [] #a list of file names that contain both PPG and ECG Lead I waveforms
for name in hd_names:
  record = wfdb.rdheader(name)
  if record.sig_len >= 9000: #extact only records contrains ECG lead I and PPG, and >=1min
    qualified_names.append(name)
print('There are total', len(qualified_names), 'qualified records')

In [0]:
#load label numpy file
df = pd.read_csv(r'C:\Users\57lzhang.US04WW4008\Desktop\Afib\Afib_ECG data\training2017\REFERENCE.csv', sep=',', header=None)
#create a new name list that only stores the file name
new_name =[]
for name in qualified_names:
  new_name.append(name[-6:])
len(new_name)

ini_labels = df[df[0].isin(new_name)][1].to_numpy()

In [0]:
##read signals
ECG_signals = [] #create a  list to store all  ECG signals
for name in qualified_names:
  record = wfdb.rdrecord(name)
  ECG_signals.append(record.p_signal)

print('ECG signals len:', len(ECG_signals))

In [0]:
ECG_signals[0].shape

In [0]:
#Write a function to split the data with 30s segment and label it
def generate_data(source,seg_len):
  n=0
  for signal in source:
    for i in int(len(signal)/seg_len):
      seg = signal[:9000*(i+1)]
      label = ini_labels[n]
      signals.append(seg)
      labels.append(label)
  n+=1
  return signals,labels

In [0]:
dataset =tf.data.Dataset.from_tensor_slices((signals,labels))

In [0]:
#create training, validation and test dataset